<a href="https://colab.research.google.com/github/kmk4444/System_engineering/blob/main/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
import csv
from openai import OpenAI
import time
import os
import random
import re

# OpenAI API anahtarınızı güvenli bir şekilde kullanmak için ortam değişkenlerini kullanın
client = OpenAI(
    # This is the default and can be omitted
    api_key='sk-proj-',
)

konular = [

]

soru_kaliplari = [

]


def generate_answer(question):
    try:
        response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": "Sen bir mühendislik uzmanısın ve sorulara detaylı ve anlaşılır cevaplar veriyorsun. Maksimum 500 token cevap verebilirsin. Bu kapsamda cümleleri yarım bıracak şekilde cevap verme."},
                {"role": "user", "content": question}
            ],
            max_tokens=500,
            n=1,
            stop=None,
            temperature=0.7,
        )
        answer = response.choices[0].message.content.strip()
        return answer
    except Exception as e:
        print(f"Bir hata oluştu: {e}")
        return ""

# Cümleler arasındaki boşlukları kaldıran fonksiyon
def remove_spaces_between_sentences(text):
    # Nokta, ünlem veya soru işaretinden sonra gelen boşlukları kaldırır
    return re.sub(r'(?<=[.!?])\s+:', '', text)

with open('dataset.csv', 'w', encoding='utf-8-sig', newline='') as csvfile:
    fieldnames = ['instruction', 'input', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for konu1 in konular:
        for soru_kalibi in soru_kaliplari:
            if '{1}' in soru_kalibi:
                # İkinci bir konu seçiyoruz
                konu2 = random.choice(konular)
                while konu2 == konu1:
                    konu2 = random.choice(konular)
                instruction = soru_kalibi.format(konu1, konu2)
            else:
                instruction = soru_kalibi.format(konu1)
            output = generate_answer(instruction)
            # Cevabı işliyoruz
            output = remove_spaces_between_sentences(output)
            writer.writerow({'instruction': instruction, 'input': '', 'output': output})
            time.sleep(1)  # API hız sınırlarını aşmamak için bekleme süresi ekleyin

print("Veri seti oluşturuldu.")


In [ ]:
len(soru_kaliplari)

214